##### Copyright 2021 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Compile TensorFlow Lite models for Coral Edge TPU

This notebook demonstrates how to take the object detection model trained with [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) and compile it to run on [Coral Edge TPU](https://coral.ai/). See this [notebook](https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb) if you want to learn how to train a custom TensorFlow Lite object detection model using Model Maker.

First, download the TensorFlow Lite model for detecting Android figurines that we have trained with Model Maker.

In [ ]:
!wget -O android.tflite https://storage.googleapis.com/download.tensorflow.org/models/tflite/android_figurine.tflite

--2022-11-10 12:54:01--  https://storage.googleapis.com/download.tensorflow.org/models/tflite/android_figurine.tflite
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 142.250.1.128, 108.177.121.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4427830 (4.2M) [application/octet-stream]
Saving to: ‘android.tflite’

android.tflite      100%[===================>]   4.22M  --.-KB/s    in 0.03s   

2022-11-10 12:54:01 (155 MB/s) - ‘android.tflite’ saved [4427830/4427830]



Then install the EdgeTPU runtime.

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  53978      0 --:--:-- --:--:-- --:--:-- 53978
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad

Then compile the TensorFlow Lite model using the Edge TPU compiler.

**Note:** When training the model using a custom dataset, beware that if your dataset includes more than 20 classes, you'll probably have slower inference speeds compared to if you have fewer classes. This is due to an aspect of the EfficientDet architecture in which a certain layer cannot compile for the Edge TPU when it carries more than 20 classes.

In [ ]:
#!edgetpu_compiler deeplabv3_mnv2_custom_257_etpu.tflite

#!edgetpu_compiler -s -d -a -v --min_runtime_version 10 deeplabv3_mnv2_custom_257_etpu.tflite
# https://coral.ai/docs/edgetpu/compiler/#compiler-and-runtime-versions

######!edgetpu_compiler --min_runtime_version 13 -s -d -a deeplabv3_mnv2_custom_257_etpu.tflite
#!edgetpu_compiler -s -d -a deeplabv3_mnv2_custom_257_test_int8.tflite


!edgetpu_compiler --min_runtime_version 13 -s -d -a deeplabv3_mnv2_257_proper_rep.tflite


Edge TPU Compiler version 16.0.384591198
Searching for valid delegate with step 1
Try to compile segment with 72 ops
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 1381 ms.

Input model: deeplabv3_mnv2_257_proper_rep.tflite
Input size: 2.46MiB
Output model: deeplabv3_mnv2_257_proper_rep_edgetpu.tflite
Output size: 3.01MiB
On-chip memory used for caching model parameters: 2.51MiB
On-chip memory remaining for caching model parameters: 4.44MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 2
Total number of operations: 72
Operation log: deeplabv3_mnv2_257_proper_rep_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edg

Finally, we'll copy the metadata, including the label file, from the original TensorFlow Lite model to the EdgeTPU model. We need to install the TensorFlow Lite Support Library to handle model metadata.

In [ ]:
!pip install -q tflite-support

     |████████████████████████████████| 60.9 MB 1.1 MB/s 
     |████████████████████████████████| 216 kB 67.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.10.26 which is incompatible.


In [ ]:
from tflite_support import metadata

populator_dst = metadata.MetadataPopulator.with_model_file('deeplabv3_mnv2_257_proper_rep_edgetpu.tflite')

with open('deeplabv3_mnv2_257_proper_rep.tflite', 'rb') as f:
  populator_dst.load_metadata_and_associated_files(f.read())

populator_dst.populate()
updated_model_buf = populator_dst.get_model_buffer()

In [ ]:
# Download the TFLite model compiled for EdgeTPU to your local computer.
from google.colab import files
files.download('deeplabv3_mnv2_257_proper_rep_edgetpu.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>